In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import HiveContext
import json
import time
import sys

content = [line.rstrip() for line in open('jobsArguments.conf')]

analysisName=content[0]
scope=content[1]
scale=content[2]
sqlControl=content[3]
sqlCase=content[4]
group1name=content[5]
group2name=content[6]

nPartitions=4
conf = (SparkConf()
         .setMaster("local["+str(nPartitions)+"]")
         .setAppName(analysisName)
#         .set("spark.executor.memory", "5g")
#         .set("spark.driver.memory", "5g")
#         .set("spark.python.worker.memory", "5g")
       )
sc.stop()
sc = SparkContext(conf=conf)


#parquetFile = sqlContext.read.parquet("/user/hive/warehouse/gvr4.db/variantsulb")
#parquetFile = sqlContext.read.parquet("/Users/yalb/Projects/Github/Docker/cdh54_4_add1000g/variants2")
#parquetFile = sqlContext.read.parquet("hdfs://127.0.0.1:8020/user/hive/warehouse/gvr.db/test")
#parquetFile = sqlContext.read.parquet("hdfs://localhost/user/hive/warehouse/gvr3.db/variants")



In [3]:
#sqlContext = HiveContext(sc) #sqlContext._get_hive_ctx() #HiveContext(sc) 
sqlContext = SQLContext(sc)
sqlContext.sql("SET spark.sql.parquet.binaryAsString=true")

#parquetFile = sqlContext.read.parquet("hdfs://node001:8020/user/hive/warehouse/highlander.db/exomes_hc")

parquetFile = sqlContext.read.parquet("/Users/yalb/Projects/Github/digest/variantsulb")
parquetFile.registerTempTable("parquetFile");


In [47]:
#analysisName="control_vs_neurodev_rare_digenic"
#group1name="control_ulb_rare_damaging"
#group2name="neurodev_ulb_rare_damaging"
scope="digenic"
#scale="gene"
#controlMAF=0.5

In [4]:
#RDDtest = sqlContext.sql("SELECT distinct patient from parquetFile")

In [5]:
#RDDtest.count()

In [31]:
#Input is vector patient, chr, pos, ref, alt, gene_symbol, zygosity
def createKey_VariantGene(variantData):
    #ID is chr:pos:ref:alt
    ID=variantData[1]+":"+str(variantData[2])+":"+variantData[3]+":"+variantData[4]
    #return ID, gene_symbol, patient, zygosity
    zygosity=2
    if variantData[6]=="Heterozygous":
        zygosity=1
    patientsID_dictionnary=patientsID_dictionnary_b.value
    patientIndex=patientsID_dictionnary[variantData[0]]
    return ((ID,variantData[5]),(patientIndex,zygosity))

def buildVariantVector(ID,variantData,patientsID):
    variantData=list(variantData)
    genotype=[0]*len(patientsID)
    
    #Get sampleID/Genotype for each variant
    for i in range(0,len(variantData)):
        genotype[variantData[i][1]]=variantData[i][2]
    
    return ((ID,variantData[0][0]),genotype)



In [5]:
#variantGeneEntry: key is (variantID,gene), value is (patientIndex,zygosity)
def geneAsKey(variantGeneEntry):    
    return (variantGeneEntry[0][1],(variantGeneEntry[0][0],variantGeneEntry[1]))

def makePairParts(k,v,nbPart):
    result=[]
    for i in range(0,nbPart):
        result.append(((k,i),v))
        
    return [(str(sorted([k,i])),(v)) for i in range(0,nbPart)]

def f(splitIndex ,v): 
    return [(splitIndex,list(v))]

In [6]:
def scoreVariantUnivariate(k,variantData):
    variantData=list(variantData)
    
    score=0
    sumControl=0
    
    sumCase=sum([int(x>0) for x in variantData[0]])
    sumControl=sum([int(x>0) for x in variantData[1]])
    
    score=sumCase#-sumControl
    if sumControl>0:
        score=0
        
    if score>0:
        return (k,(score,sumCase,sumControl))

In [7]:
def getGenotypeVectorByGene(gene_symbol,variantList,patientsID_dictionnary,patientsID_split_index):
    genoSum=[0]*len(patientsID_dictionnary)
    
    if len(variantList)>0:
        #Go through list of variants
        for i in range(0,len(variantList)):
            #Get variant ID, and list of sample_index,genotype
            (variantID,sample_geno_list)=variantList[i]
            sample_geno_list=list(sample_geno_list)
            
            #Go through list of sample_index,genotype
            for j in range(0,len(sample_geno_list)):
                genoSum[sample_geno_list[j][0]]=genoSum[sample_geno_list[j][0]]+sample_geno_list[j][1]
    return genoSum

In [40]:
#variantList is [(locusID,[sample_index,genotype])]
def scoreGene(gene_symbol,variantList):
    variantList=list(variantList)
    
    patientsID_dictionnary=patientsID_dictionnary_b.value
    genoSum=[0]*len(patientsID_dictionnary)
    
    patientsID_split_index=patientsID_split_index_b.value
    
    genoSum=getGenotypeVectorByGene(gene_symbol,variantList,patientsID_dictionnary,patientsID_split_index)
    
    sumCase=sum([int(x>0) for x in genoSum[0:patientsID_split_index]])
    sumControl=sum([int(x>0) for x in genoSum[(patientsID_split_index+1):len(patientsID_dictionnary)]])
    score=sumCase-sumControl

    if score>0:
        return (gene_symbol,(score,sumCase,sumControl))

In [9]:
def scoreGenePair(gene_symbol_pair,variantList):
    
    variantList=list(variantList)
    
    patientsID_dictionnary=patientsID_dictionnary_b.value
    
    patientsID_split_index=patientsID_split_index_b.value
    
    score=0
    if len(variantList)==2:
        (genes,variantList1)=variantList[0]
        (genes,variantList2)=variantList[1]
        
        gene1=genes[0]
        gene2=genes[1]
        
        variantList1=list(variantList1)
        variantList2=list(variantList2)
        
        genoSum1=getGenotypeVectorByGene(gene1,variantList1,patientsID_dictionnary,patientsID_split_index)
        genoSum2=getGenotypeVectorByGene(gene2,variantList2,patientsID_dictionnary,patientsID_split_index)
        
        genoSum=genoSum1+genoSum2
        
        sumCase=sum([int(x>0) for x in genoSum[0:patientsID_split_index]])
        sumControl=sum([int(x>0) for x in genoSum[(patientsID_split_index+1):len(patientsID_dictionnary)]])
        
        score=sumCase-sumControl

        if score>0:
            return (gene_symbol_pair,((gene1,gene2),score,sumCase,sumControl))

#Key is (variantID, gene)
def getGene(variantGene_key):
    gene=variantGene_key[1]
    
    return (gene)

def createPairsGenes(k,v,genes):
    return [(str(sorted([k,gene])),(sorted([k,gene]),v)) for gene in genes]

def fillMissing(k,v):
    v=list(v)
    if v[0] is None:
        v[0]=[0]*len(sample_id_case_b.value)
    if v[1] is None:
        v[1]=[0]*len(sample_id_control_b.value)
        
    return (k,v)

def fillMissing(k,v):
    v=list(v)
    if v[1] is None:
        v[1]=[0]*len(dict_patient_control_b.value)
        
    return (k,v)

In [32]:
start_time = time.time()

variants_case = sqlContext.sql("SELECT patient,chr,pos,reference,alternative,gene_symbol,zygosity FROM parquetFile "+sqlCase)
patientsID_case=sorted(variants_case.map(lambda v:v[0]).distinct().collect())

if sqlControl!="NULL":
    variants_control= sqlContext.sql("SELECT patient,chr,pos,reference,alternative,gene_symbol,zygosity FROM parquetFile "+sqlControl)
#    controlMAF=float(controlMAF)
else:
    variants_control=sc.emptyRDD()
#    controlMAF=0   
patientsID_control=sorted(variants_control.map(lambda v:v[0]).distinct().collect())

patientsID=patientsID_case+patientsID_control
patientsID_dictionnary=dict(zip(patientsID,range(len(patientsID))))

patientsID_split_index_b=sc.broadcast(len(patientsID_case))

patientsID_dictionnary_b = sc.broadcast(patientsID_dictionnary)

variants=variants_control.unionAll(variants_case)

variants_grouped=variants.map(createKey_VariantGene).groupByKey()


In [48]:
#start_time = time.time()

if scope=='monogenic':
    if scale=='variant':
        scores=genoMat.map(lambda (k,v):scoreVariantUnivariate(k,v)).filter(lambda x:x is not None).takeOrdered(10000000, key=lambda (k,(v1,v2,v3)): -v1)

    if scale=='gene':
        scores=variants_grouped.map(geneAsKey).groupByKey().map(lambda (k,v):scoreGene(k,v)).filter(lambda x:x is not None).takeOrdered(1000, key=lambda (k,(v1,v2,v3)): -v1)
    
if scope=='digenic':
    variants_grouped_by_gene=variants_grouped.map(geneAsKey).groupByKey()#.flatMap(lambda (k,v):createPairsGenes(k,v,genes)).groupByKey().map(lambda (k,v):scoreGenePair(k,v))#.filter(lambda x:x is not None).takeOrdered(1000, key=lambda (k,(genes,v1,v2,v3)): -v1)
    genes=variants_grouped_by_gene.keys().collect()
    scores=variants_grouped_by_gene.flatMap(lambda (k,v):createPairsGenes(k,v,genes)).groupByKey().map(lambda (k,v):scoreGenePair(k,v)).filter(lambda x:x is not None).takeOrdered(1000, key=lambda (k,(genes,v1,v2,v3)): -v1)
    
end_time=time.time()
runtime=end_time - start_time
print(runtime)


222.242046833


In [35]:
scores=variants_grouped.map(geneAsKey).groupByKey().filter(lambda x:x is not None).filter(lambda (k,v):k=="SREBF1").map(lambda (k,v):scoreGene(k,v))

In [49]:
scores[0:20]

[("[u'CACNA1B', u'PRR21']", ((u'CACNA1B', u'PRR21'), 2, 10, 8)),
 ("[u'OR5P3', u'RPS3A']", ((u'OR5P3', u'RPS3A'), 2, 2, 0)),
 ("[u'ADH6', u'KIAA0513']", ((u'ADH6', u'KIAA0513'), 2, 2, 0)),
 ("[u'ADH6', u'HRCT1']", ((u'ADH6', u'HRCT1'), 2, 2, 0)),
 ("[u'KRT77', u'OSGEPL1']", ((u'KRT77', u'OSGEPL1'), 2, 2, 0)),
 ("[u'ADH6', u'OR11H4']", ((u'ADH6', u'OR11H4'), 2, 2, 0)),
 ("[u'GABPB1', u'WDR89']", ((u'GABPB1', u'WDR89'), 2, 2, 0)),
 ("[u'ADH6', u'TPTE']", ((u'ADH6', u'TPTE'), 2, 2, 0)),
 ("[u'MUC3A', u'RPS3A']", ((u'MUC3A', u'RPS3A'), 2, 2, 0)),
 ("[u'CACNA1B', u'TTC31']", ((u'CACNA1B', u'TTC31'), 2, 10, 8)),
 ("[u'ADH6', u'ZNF99']", ((u'ADH6', u'ZNF99'), 2, 2, 0)),
 ("[u'ADH6', u'CEP170P1']", ((u'ADH6', u'CEP170P1'), 2, 2, 0)),
 ("[u'KIAA0040', u'KRT77']", ((u'KIAA0040', u'KRT77'), 2, 2, 0)),
 ("[u'AC068987.1', u'ADH6']", ((u'AC068987.1', u'ADH6'), 2, 2, 0)),
 ("[u'THNSL2', u'WDR89']", ((u'THNSL2', u'WDR89'), 2, 2, 0)),
 ("[u'ADH6', u'MAP7']", ((u'ADH6', u'MAP7'), 2, 2, 0)),
 ("[u'AP0003

In [50]:
analysisName="neurodev_vs_control_digenic_rare_high"

In [51]:
scores=[analysisName,scale,scope,start_time,end_time,runtime,scores,patientsID_case,patientsID_control,group1name,group2name]

with open(analysisName+'.txt', 'w') as outfile:
    json.dump(scores, outfile)
    

In [ ]:
sc.stop()